Segmentation d'images avec U-Net
on va  construire mon propre U-Net, un type de CNN conçu pour une segmentation d'image rapide et précise, et l'utiliser pour prédire une étiquette pour chaque pixel d'une image - dans ce cas, une image d'une poummon humain (covid et normal) base de données.

---

Ce type de classification d'images est appelé segmentation sémantique d'images. C'est similaire à la détection d'objets .
 la segmentation sémantique de l'image vous permet de prédire un masque précis pour chaque objet de l'image en étiquetant chaque pixel de l'image avec sa classe correspondante. Le mot "sémantique" fait ici référence à ce qui est affiché, ainsi par exemple la classe "poumon" est indiquée ci-dessous par le masque jaunne et "le reste " est indiquée par un masque violet.

---



In [ ]:
!pip install -q kaggle 


In [ ]:
!mkdir ~/.kaggle/ 


In [ ]:
! cp '/content/drive/MyDrive/kaggle/kaggle.json' ~/.kaggle/ 


In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
! kaggle datasets list

In [ ]:
! kaggle datasets download -d tawsifurrahman/covid19-radiography-database

100% 775M/778M [00:06<00:00, 174MB/s]
100% 778M/778M [00:06<00:00, 126MB/s]


In [ ]:
!unzip /content/covid19-radiography-database.zip -d database-data

Streaming output truncated to the last 5000 lines.
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7921.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7922.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7923.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7924.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7925.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7926.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7927.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7928.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7929.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-793.png  
  inflating: database-data/COVID-19_Radiography_Dataset/Normal/masks/Normal-7930

In [ ]:
!pip install "jax<=0.3.16" "jaxlib<=0.3.16"


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.0 MB 7.1 MB/s 
  Created wheel for jax: filename=jax-0.3.16-py3-none-any.whl size=1197715 sha256=baa3b6cf48d6f34e70a1e8d542e485a17fcc6548124e64b3f1eea60915b2e915
  Stored in directory: /root/.cache/pip/wheels/b9/0d/a2/2091279bf29b90a2211a271f6f006ef13a656211a9e69ba1e3
Successfully built jax
  Attempting uninstall: jax
    Found existing installation: jax 0.3.17
    Uninstalling jax-0.3.17:
      Successfully uninstalled jax-0.3.17


In [ ]:
# there are 2 categories in COVID-19_Radiography_Dataset
target_labels = [
                 "Normal", 
                 "COVID",
                 "masks",
                 "images"
                 
               ]

In [ ]:
path = "/content/covid19-radiography-database/"


In [ ]:
!pip install tensorflow==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 668.3 MB 20 kB/s 
     |████████████████████████████████| 462 kB 9.1 MB/s 
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.8.2+zzzcolab20220719082949
    Uninstalling tensorflow-2.8.2+zzzcolab20220719082949:
      Successfully uninstalled tensorflow-2.8.2+zzzcolab20220719082949


In [ ]:
!pip install -U git+https://github.com/albu/albumentations --no-cache-dir


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/albu/albumentations to /tmp/pip-req-build-hglbxpyy
  Running command git clone -q https://github.com/albu/albumentations /tmp/pip-req-build-hglbxpyy
  Created wheel for albumentations: filename=albumentations-1.3.0-py3-none-any.whl size=123546 sha256=45fd66becbd6dc90b6bbadd7cfdb44e0cacd390c9cec60469f9074eaffe5384c
  Stored in directory: /tmp/pip-ephem-wheel-cache-oc2yw7jq/wheels/63/11/1a/c77caf3ae9b9b6d57b3ee5e6a41a50f3bc12c66a70f6b90bf0
Successfully built albumentations
  Attempting uninstall: albumentations
    Found existing installation: albumentations 1.2.1
    Uninstalling albumentations-1.2.1:
      Successfully uninstalled albumentations-1.2.1


In [ ]:
pip install segmentation_models_pytorch


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 97 kB 4.8 MB/s 
     |████████████████████████████████| 376 kB 46.7 MB/s 
     |████████████████████████████████| 58 kB 7.7 MB/s 
  Created wheel for efficientnet-pytorch: filename=efficientnet_pytorch-0.7.1-py3-none-any.whl size=16446 sha256=0fe744def389c629f9d5d78fe1d37f35d228f3612e93c47041aa1b1de5b0ddc1
  Stored in directory: /root/.cache/pip/wheels/0e/cc/b2/49e74588263573ff778da58cc99b9c6349b496636a7e165be6
  Created wheel for pretrainedmodels: filename=pretrainedmodels-0.7.4-py3-none-any.whl size=60965 sha256=5151776cd26a8ec33559bfe2f0c685e14ba387c2560e3ff912eaacdccee494da
  Stored in directory: /root/.cache/pip/wheels/ed/27/e8/9543d42de2740d3544db96aefef63bda3f2c1761b3334f4873
Successfully built efficientnet-pytorch pretrainedmodels


In [ ]:
!pip install wandb -qqq
import wandb

     |████████████████████████████████| 1.8 MB 9.0 MB/s 
     |████████████████████████████████| 181 kB 45.0 MB/s 
     |████████████████████████████████| 162 kB 84.2 MB/s 
     |████████████████████████████████| 63 kB 1.7 MB/s 
     |████████████████████████████████| 162 kB 78.6 MB/s 
     |████████████████████████████████| 158 kB 78.7 MB/s 
     |████████████████████████████████| 157 kB 60.0 MB/s 
     |████████████████████████████████| 157 kB 77.5 MB/s 
     |████████████████████████████████| 157 kB 101.4 MB/s 
     |████████████████████████████████| 157 kB 73.0 MB/s 
     |████████████████████████████████| 157 kB 71.9 MB/s 
     |████████████████████████████████| 157 kB 78.5 MB/s 
     |████████████████████████████████| 157 kB 68.3 MB/s 
     |████████████████████████████████| 156 kB 77.5 MB/s 


In [ ]:
pip install opencv-python-headless==4.1.2.30


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.8 MB 1.1 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [ ]:
!pip install numpy==1.21.6

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install tf-estimator-nightly==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tf-estimator-nightly==2.8.0 (from versions: 1.12.0.dev20181101, 1.12.0.dev20181105, 1.12.0.dev20181114, 1.12.0.dev20181115, 1.12.0.dev20181116, 1.12.0.dev20181117, 1.12.0.dev20181118, 1.12.0.dev20181119, 1.12.0.dev20181120, 1.12.0.dev20181121, 1.12.0.dev20181122, 1.12.0.dev20181123, 1.12.0.dev20181124, 1.12.0.dev20181125, 1.12.0.dev20181126, 1.12.0.dev20181127, 1.12.0.dev20181128, 1.12.0.dev20181129, 1.12.0.dev20181201, 1.12.0.dev20181202, 1.12.0.dev20181203, 1.12.0.dev20181204, 1.12.0.dev20181205, 1.12.0.dev20181206, 1.12.0.dev20181207, 1.12.0.dev20181208, 1.12.0.dev20181214, 1.12.0.dev20181215, 1.12.0.dev20181216, 1.12.0.dev20181217, 1.13.0.dev2019010910, 1.13.0.dev2019011813, 1.13.0.dev2019011901, 1.13.0.dev2019012001, 1.13.0.dev2019012101, 1.13.0.dev2019012201, 1.13.0.dev2019012415, 1.13.0.dev2019012416, 1.13.0.dev20190125

1 - Packages
 importer toutes les bibliothèques dont vous aurez besoin :

In [ ]:
import math
import numpy as np
import h5py

import matplotlib.pyplot as plt
from matplotlib.pyplot import imread
import scipy
from PIL import Image
import pandas as pd
import tensorflow as tf
import tensorflow.keras.layers as tfl
from tensorflow.python.framework import ops




In [ ]:
!pip install keras==2.8.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install test-utils


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for test-utils: filename=test_utils-0.1.0-py3-none-any.whl size=3852 sha256=5d955f04a99a36e2fcf64e9b974568844a80e86907ee3bf4667a8e6e44d5cbeb
  Stored in directory: /root/.cache/pip/wheels/41/f0/b5/32f8bbb837d4c7b2aa6e2881685f5b4e370d92f9ef84a7d74d
Successfully built test-utils


In [ ]:
!pip install summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for summary: filename=summary-0.2.0-py3-none-any.whl size=18140 sha256=fcf9a5fcbe41b042b72582a8a5d21921c2b344afed9f0eea7116569f73c07422
  Stored in directory: /root/.cache/pip/wheels/0b/01/32/4a6935e408ea2e092cea949e4534d4e3528a0a6cdf2f08ea5b
Successfully built summary


In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate
from tensorflow.python.summary import summary
from tensorflow.keras.utils import to_categorical 
from typing import Callable, Optional



In [ ]:
import test_utils

In [ ]:
import numpy as np
from termcolor import colored

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate

# Compare the two inputs
def comparator(learner, instructor):
    if len(learner) != len(instructor):
        raise AssertionError(f"The number of layers in the proposed model does not agree with the expected model: expected {len(instructor)}, got {len(learner)}.") 
    for a, b in zip(learner, instructor):
        if tuple(a) != tuple(b):
            print(colored("Test failed", attrs=['bold']),
                  "\n Expected value \n\n", colored(f"{b}", "green"), 
                  "\n\n does not match the input value: \n\n", 
                  colored(f"{a}", "red"))
            raise AssertionError("Error in test") 
    print(colored("All tests passed!", "green"))

# extracts the description of a given model
def summary(model):
    model.compile(optimizer='adam',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    result = []
    for layer in model.layers:
        descriptors = [layer.__class__.__name__, layer.output_shape, layer.count_params()]
        if (type(layer) == Conv2D):
            descriptors.append(layer.padding)
            descriptors.append(layer.activation.__name__)
            descriptors.append(layer.kernel_initializer.__class__.__name__)
        if (type(layer) == MaxPooling2D):
            descriptors.append(layer.pool_size)
        if (type(layer) == Dropout):
            descriptors.append(layer.rate)
        result.append(descriptors)
    return result

def datatype_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += datatype_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} in variable {}. Got {} but expected type {}".format(error,
                                                                          key, type(target_output[key]), type(expected_output[key])))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += datatype_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} in variable {}, expected type: {}  but expected type {}".format(error,
                                                                          i, type(target_output[i]), type(expected_output[i])))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        assert isinstance(target_output, type(expected_output))
        return 1
            
def equation_output_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += equation_output_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error,
                                                                          key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += equation_output_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable in position {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            np.testing.assert_array_almost_equal(target_output, expected_output)
        else:
            assert target_output == expected_output
        return 1
    
def shape_check(expected_output, target_output, error):
    success = 0
    if isinstance(target_output, dict):
        for key in target_output.keys():
            try:
                success += shape_check(expected_output[key], 
                                         target_output[key], error)
            except:
                print("Error: {} for variable {}.".format(error, key))
        if success == len(target_output.keys()):
            return 1
        else:
            return 0
    elif isinstance(target_output, tuple) or isinstance(target_output, list):
        for i in range(len(target_output)):
            try: 
                success += shape_check(expected_output[i], 
                                         target_output[i], error)
            except:
                print("Error: {} for variable {}.".format(error, i))
        if success == len(target_output):
            return 1
        else:
            return 0
                
    else:
        if hasattr(target_output, 'shape'):
            assert target_output.shape == expected_output.shape
        return 1
                
def single_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            if test_case['name'] == "datatype_check":
                assert isinstance(target(*test_case['input']),
                                  type(test_case["expected"]))
                success += 1
            if test_case['name'] == "equation_output_check":
                assert np.allclose(test_case["expected"],
                                   target(*test_case['input']))
                success += 1
            if test_case['name'] == "shape_check":
                assert test_case['expected'].shape == target(*test_case['input']).shape
                success += 1
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))
        
def multiple_test(test_cases, target):
    success = 0
    for test_case in test_cases:
        try:
            target_answer = target(*test_case['input'])                   
            if test_case['name'] == "datatype_check":
                success += datatype_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "equation_output_check":
                success += equation_output_check(test_case['expected'], target_answer, test_case['error'])
            if test_case['name'] == "shape_check":
                success += shape_check(test_case['expected'], target_answer, test_case['error'])
        except:
            print("Error: " + test_case['error'])
            
    if success == len(test_cases):
        print("\033[92m All tests passed.")
    else:
        print('\033[92m', success," Tests passed")
        print('\033[91m', len(test_cases) - success, " Tests failed")
        raise AssertionError("Not all tests were passed for {}. Check your equations and avoid using global variables inside the function.".format(target.__name__))
        

In [ ]:
pip install tensorflow-gpu==2.4


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 394.7 MB 21 kB/s 
     |████████████████████████████████| 3.8 MB 58.1 MB/s 
     |████████████████████████████████| 14.8 MB 65.8 MB/s 
     |████████████████████████████████| 462 kB 100.6 MB/s 
     |████████████████████████████████| 2.9 MB 44.9 MB/s 
     |████████████████████████████████| 132 kB 54.3 MB/s 
  Created wheel for wrapt: filename=wrapt-1.12.1-cp37-cp37m-linux_x86_64.whl size=68720 sha256=9a19f00f37301942b2034430bd5e0e013411143e0cbeb245756dbb9cef6b85f4
  Stored in directory: /root/.cache/pip/wheels/62/76/4c/aa25851149f3f6d9785f6c869387ad82b3fd37582fa8147ac6
Successfully built wrapt
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Uninstalling typing-extensions-4.1.1:
      Successfully uninstalled typing-extensions-4.1.1
  Attempting uninstall: numpy
    Found existing installation: 

2 - Charger et diviser les données:

In [ ]:
import os
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import imageio

import matplotlib.pyplot as plt
%matplotlib inline

path = '/database-data/COVID-19_Radiography_Dataset/'
image_path = os.path.join(path, './database-data/COVID-19_Radiography_Dataset/COVID/images/')
mask_path = os.path.join(path, './database-data/COVID-19_Radiography_Dataset/COVID/masks/')
image_list = os.listdir(image_path)
mask_list = os.listdir(mask_path)
image_list = [image_path+i for i in image_list]
mask_list = [mask_path+i for i in mask_list]

FileNotFoundError: ignored

Découvrez quelques-unes des images non masquées et masquées de l'ensemble de données :
Une fois l'exploration terminée, revenez à N=1 par exemple. Sinon, l'autograder lancera une erreur d'index de liste hors plage.

In [ ]:
N = 1
img = imageio.imread(image_list[N])
mask = imageio.imread(mask_list[N])
#mask = np.array([max(mask[i, j]) for i in range(mask.shape[0]) for j in range(mask.shape[1])]).reshape(img.shape[0], img.shape[1])

fig, arr = plt.subplots(1, 2, figsize=(14, 10))
arr[0].imshow(img)
arr[0].set_title('Image')
arr[1].imshow(mask[:, :, 0])
arr[1].set_title('Segmentation')

2.1 - Divisez votre ensemble de données en images non masquées et masquées

In [ ]:
import tensorflow as tf

image_list_ds = tf.data.Dataset.list_files(image_list, shuffle=False)
mask_list_ds = tf.data.Dataset.list_files(mask_list, shuffle=False)

for path in zip(image_list_ds.take(3), mask_list_ds.take(3)):
    print(path)

In [ ]:
image_filenames = tf.constant(image_list)
masks_filenames = tf.constant(mask_list)

dataset = tf.data.Dataset.from_tensor_slices((image_filenames, masks_filenames))

for image, mask in dataset.take(1):
    print(image)
    print(mask)

2.2 - Prétraitez vos données

In [ ]:
def process_path(image_path, mask_path):
    img = tf.io.read_file(image_path)
    img = tf.image.decode_png(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)

    mask = tf.io.read_file(mask_path)
    mask = tf.image.decode_png(mask, channels=3)
    mask = tf.math.reduce_max(mask, axis=-1, keepdims=True)
    return img, mask

def preprocess(image, mask):
    input_image = tf.image.resize(image, (96, 128), method='nearest')
    input_mask = tf.image.resize(mask, (96, 128), method='nearest')

    input_image = input_image / 255.

    return input_image, input_mask

image_ds = dataset.map(process_path)
processed_image_ds = image_ds.map(preprocess)

In [ ]:
image_ds = dataset.map(process_path)


In [ ]:
image_ds


In [ ]:
processed_image_ds = image_ds.map(preprocess)


In [ ]:
processed_image_ds


3 - U-Net
U-Net, du nom de sa forme en U, a été créé à l'origine en 2015 pour la détection des tumeurs, mais dans les années qui ont suivi, il est devenu un choix très populaire pour d'autres tâches de segmentation sémantique.

U-Net s'appuie sur une architecture précédente appelée le réseau entièrement convolutif, ou FCN, qui remplace les couches denses trouvées dans un CNN typique par une couche de convolution transposée qui suréchantillonne la carte des caractéristiques à la taille de l'image d'entrée d'origine, tout en préservant le informations spatiales. Cela est nécessaire car les couches denses détruisent les informations spatiales (le "où" de l'image), qui sont une partie essentielle des tâches de segmentation d'image. Un avantage supplémentaire de l'utilisation des convolutions de transposition est que la taille d'entrée n'a plus besoin d'être fixe, comme c'est le cas lorsque des couches denses sont utilisées.

Malheureusement, la dernière couche de fonctionnalités du FCN souffre d'une perte d'informations due à un sous-échantillonnage trop important. Il devient alors difficile de suréchantillonner une fois que tant d'informations ont été perdues, ce qui provoque une sortie qui semble grossière.

U-Net améliore le FCN, en utilisant une conception quelque peu similaire, mais différant à certains égards importants. Au lieu d'une convolution transposée à la fin du réseau, il utilise un nombre correspondant de convolutions pour sous-échantillonner l'image d'entrée en une carte de caractéristiques, et des convolutions transposées pour suréchantillonner ces cartes jusqu'à la taille d'image d'entrée d'origine. Il ajoute également des connexions de saut, pour conserver les informations qui seraient autrement perdues lors de l'encodage. Les connexions de saut envoient des informations à chaque couche de suréchantillonnage dans le décodeur à partir de la couche de sous-échantillonnage correspondante dans l'encodeur, capturant des informations plus fines tout en maintenant un calcul bas. Ceux-ci aident à prévenir la perte d'informations, ainsi que le surajustement du modèle.

chemin de contraction:(Encoder containing downsampling steps):

730 / 5 000
Résultats de traduction
Les images sont d'abord alimentées à travers plusieurs couches convolutionnelles qui réduisent la hauteur et la largeur, tout en augmentant le nombre de canaux.
---
Le chemin contractuel suit une architecture CNN régulière, avec des couches convolutives, le

```
# `Ce texte est au format code`
```

urs activations et des couches de regroupement( pooling) pour sous-échantillonner l'image et extraire ses caractéristiques. En détail, il consiste en l'application répétée de deux convolutions non rembourrées 3 x 3, chacune suivie d'une unité linéaire rectifiée (ReLU) et d'une opération de mise en commun max (max pooling )2 x 2 avec pas 2 pour le sous-échantillonnage. À chaque étape de sous-échantillonnage(downsampling), le nombre de canaux de caractéristiques est doublé.
---
Fonction de recadrage : cette étape recadre l'image du chemin de contraction et la concatène à l'image actuelle sur le chemin d'expansion pour créer une connexion de saut.


Chemin d'expansion (décodeur contenant des étapes de suréchantillonnage) :

---
Le chemin d'expansion effectue l'opération inverse du chemin de contraction, agrandissant l'image à sa taille d'origine, tout en rétrécissant progressivement les canaux.

---
En détail, chaque étape du chemin d'expansion suréchantillonne la carte de caractéristiques, suivie d'une convolution 2 x 2 (la convolution transposée). Cette convolution transposée réduit de moitié le nombre de canaux de caractéristiques, tout en augmentant la hauteur et la largeur de l'image.

---
Vient ensuite une concaténation avec la carte de caractéristiques recadrée en conséquence à partir du chemin de contraction, et deux convolutions 3 x 3, chacune suivie d'un ReLU. Vous devez effectuer un recadrage pour gérer la perte de pixels de bordure dans chaque circonvolution.



---
Bloc de mappage de caractéristiques final : dans la couche finale, une convolution 1x1 est utilisée pour mapper chaque vecteur de caractéristiques à 64 composants au nombre de classes souhaité. Les dimensions du canal de la couche précédente correspondent au nombre de filtres utilisés, donc lorsque vous utilisez des convolutions 1x1, vous pouvez transformer cette dimension en choisissant un nombre approprié de filtres 1x1. Lorsque cette idée est appliquée à la dernière couche, vous pouvez réduire les dimensions du canal pour avoir une couche par classe.

Le réseau U-Net compte 23 couches convolutionnelles au total.




3.2 - Encoder (Downsampling Block)

L'encodeur est une pile de différents conv_blocks :

Chaque conv_block() est composé de 2 couches Conv2D avec des activations ReLU. Nous appliquerons Dropout et MaxPooling2D à certains conv_blocks, comme vous le vérifierez dans les sections suivantes, en particulier aux deux derniers blocs du sous-échantillonnage(downsampling.).

---
La fonction renverra deux tenseurs :

* next_layer : Cela ira dans le bloc suivant.
* skip_connection : Cela ira dans le bloc de décodage correspondant.

---
Remarque : Si max_pooling=True, le next_layer sera la sortie de la couche MaxPooling2D, mais le skip_connection sera la sortie de la couche précédemment appliquée (Conv2D ou Dropout, selon le cas). Sinon, les deux résultats seront identiques.

---
conv_block: 
-Implémenter conv_block(...). Voici les instructions pour chaque étape du conv_block, ou bloc contractuel :

-Ajoutez 2 couches Conv2D avec des filtres n_filters avec kernel_size défini sur 3, kernel_initializer défini sur 'he_normal', remplissage défini sur 'same' et activation 'relu'.

*si dropout_prob > 0, alors ajoutez une couche Dropout avec le paramètre dropout_prob
*Si max_pooling est défini sur True, ajoutez une couche MaxPooling2D avec une taille de pool de 2x2








In [ ]:
# UNQ_C1
# GRADED FUNCTION: conv_block
def conv_block(inputs=None, n_filters=32, dropout_prob=0, max_pooling=True):
    """
    Convolutional downsampling block
    
    Arguments:
        inputs -- Input tensor
        n_filters -- Number of filters for the convolutional layers
        dropout_prob -- Dropout probability
        max_pooling -- Use MaxPooling2D to reduce the spatial dimensions of the output volume
    Returns: 
        next_layer, skip_connection --  Next layer and skip connection outputs
    """

    ### START CODE HERE
    conv = Conv2D(n_filters, # Number of filters
                  3,   # Kernel size   
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(inputs)
    conv = Conv2D(n_filters, # Number of filters
                  3,   # Kernel size
                  activation='relu',
                  padding='same',
                  kernel_initializer='he_normal')(conv)
    ### END CODE HERE
    
    # if dropout_prob > 0 add a dropout layer, with the variable dropout_prob as parameter
    if dropout_prob > 0:
         ### START CODE HERE
        conv = Dropout(dropout_prob)(conv)
         ### END CODE HERE
         
        
    # if max_pooling is True add a MaxPooling2D with 2x2 pool_size
    if max_pooling:
        ### START CODE HERE
        next_layer = MaxPooling2D(pool_size=(2, 2))(conv)
        ### END CODE HERE
        
    else:
        next_layer = conv
        
    skip_connection = conv
    
    return next_layer, skip_connection

In [ ]:
from tqdm import tqdm


In [ ]:
input_size=(96, 128, 3)
n_filters = 32
inputs = Input(input_size)
cblock1 = conv_block(inputs, n_filters * 1)
model1 = tf.keras.Model(inputs=inputs, outputs=cblock1)

output1 = [['InputLayer', [(None, 96, 128, 3)], 0],
            ['Conv2D', (None, 96, 128, 32), 896, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 96, 128, 32), 9248, 'same', 'relu', 'HeNormal'],
            ['MaxPooling2D', (None, 48, 64, 32), 0, (2, 2)]]

print('Block 1:')
for layer in summary(model1):
    print(layer)

comparator(summary(model1), output1)

inputs = Input(input_size)
cblock1 = conv_block(inputs, n_filters * 32, dropout_prob=0.1, max_pooling=True)
model2 = tf.keras.Model(inputs=inputs, outputs=cblock1)

output2 = [['InputLayer', [(None, 96, 128, 3)], 0],
            ['Conv2D', (None, 96, 128, 1024), 28672, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 96, 128, 1024), 9438208, 'same', 'relu', 'HeNormal'],
            ['Dropout', (None, 96, 128, 1024), 0, 0.1],
            ['MaxPooling2D', (None, 48, 64, 1024), 0, (2, 2)]]
           
print('\nBlock 2:')   
for layer in summary(model2):
    print(layer)
    
comparator(summary(model2), output2)

In [ ]:
import tensorflow as tf
import numpy as np

from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Dropout 
from tensorflow.keras.layers import Conv2DTranspose
from tensorflow.keras.layers import concatenate


In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from sklearn.model_selection import train_test_split 
from sklearn.metrics import f1_score
import math
import cv2 as cv
import copy
from PIL import Image
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Model

from tensorflow.keras import layers
from tensorflow.keras import callbacks
from tensorflow.keras import backend as K
from tensorflow.keras.utils import Sequence
from tensorflow.keras import optimizers
import pylab
from sklearn.metrics import accuracy_score
import h5py
import skimage
from skimage import transform
from tensorflow.keras.utils import CustomObjectScope
from tensorflow.keras.models import load_model
import imageio
from scipy import ndimage
from scipy.ndimage.interpolation import map_coordinates
from skimage.measure import label

3.3 - Decoder (Upsampling Block):

Le décodeur, ou bloc de suréchantillonnage(upsampling block,), suréchantillonne les entités à la taille d'origine de l'image. À chaque niveau de suréchantillonnage, vous prendrez la sortie du bloc d'encodeur correspondant et la concaténerez avant d'alimenter le bloc de décodeur suivant.
---
Il y a deux nouveaux composants dans le décodeur : up et merge. Ce sont la convolution de transposition et les connexions de saut. De plus, il existe deux autres couches convolutives définies sur les mêmes paramètres que dans l'encodeur.

Ici, vous rencontrerez la couche Conv2DTranspose, qui effectue l'inverse de la couche Conv2D.

Implémentez upsampling_block(...).

Pour la fonction upsampling_block :

-Prend les arguments expansive_input (qui est le tenseur d'entrée de la couche précédente) et contractive_input (le tenseur d'entrée de la couche de saut précédente).

-Le nombre de filtres ici est le même que dans le bloc de sous-échantillonnage que vous avez complété précédemment.

-Votre couche Conv2DTranspose prendra n_filters avec la forme (3,3) et une foulée de (2,2), avec le rembourrage défini sur le même. Il est appliqué à expansive_input, ou au tenseur d'entrée de la couche précédente.

Ce bloc est également l'endroit où vous concaténerez les sorties des blocs d'encodeur, créant des connexions de saut.

Concaténez la sortie de votre couche Conv2DTranspose à l'entrée contractive, avec un axe de 3. En général, vous pouvez concaténer les tenseurs dans l'ordre que vous préférez. Mais pour la niveleuse, il est important que vous utilisiez [up, contractive_input]

Pour le composant final, définissez les paramètres de deux couches Conv2D sur les mêmes valeurs que celles que vous avez définies pour les deux couches Conv2D dans l'encodeur (activation ReLU, initialiseur normal He, même rembourrage).




In [ ]:
!pip install utils


In [ ]:
!pip install test_utils

In [ ]:
!pip install summary
!pip install comparator
!pip install test_utils


In [ ]:
!pip install comparator

In [ ]:
!pip install summary

In [ ]:
!pip install test-utils


In [ ]:
pip install --upgrade edc-test-utils

In [ ]:

# UNQ_C2
# GRADED FUNCTION: upsampling_block
def upsampling_block(expansive_input, contractive_input, n_filters=32):
    """
    Convolutional upsampling block
    
    Arguments:
        expansive_input -- Input tensor from previous layer
        contractive_input -- Input tensor from previous skip layer
        n_filters -- Number of filters for the convolutional layers
    Returns: 
        conv -- Tensor output
    """
    
    ### START CODE HERE
    up = Conv2DTranspose(
                 n_filters,    # number of filters
                 3,    # Kernel size
                 strides=2,
                 padding='same')(expansive_input)
    
    # Merge the previous output and the contractive_input
    merge = concatenate([up, contractive_input], axis=3)
    conv = Conv2D(n_filters,   # Number of filters
                 3,     # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(merge)
    conv = Conv2D(n_filters,  # Number of filters
                 3,   # Kernel size
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(conv)
    ### END CODE HERE
    
    return conv

In [ ]:
input_size1=(12, 16, 256)
input_size2 = (24, 32, 128)
n_filters = 32
expansive_inputs = Input(input_size1)
contractive_inputs =  Input(input_size2)
cblock1 = upsampling_block(expansive_inputs, contractive_inputs, n_filters * 1)
model1 = tf.keras.Model(inputs=[expansive_inputs, contractive_inputs], outputs=cblock1)

output1 = [['InputLayer', [(None, 12, 16, 256)], 0],
            ['Conv2DTranspose', (None, 24, 32, 32), 73760],
            ['InputLayer', [(None, 24, 32, 128)], 0],
            ['Concatenate', (None, 24, 32, 160), 0],
            ['Conv2D', (None, 24, 32, 32), 46112, 'same', 'relu', 'HeNormal'],
            ['Conv2D', (None, 24, 32, 32), 9248, 'same', 'relu', 'HeNormal']]



In [ ]:
print('Block 1:')
for layer in summary(model1):
    print(layer)

comparator(summary(model1), output1)

In [ ]:
from tensorflow import keras
from keras_preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras import regularizers, optimizers
from keras.models import Sequential
from keras import optimizers
from keras.optimizers import rmsprop_v2, adadelta_v2

3.4 - Construire le modèle

C'est là que vous allez tout assembler, en enchaînant l'encodeur, le goulot d'étranglement et le décodeur ! Vous devrez spécifier le nombre de canaux de sortie, qui pour cet ensemble particulier serait de 23. En effet, il existe 23 étiquettes possibles pour chaque pixel dans cet ensemble de données de voiture autonome.

---
unet_model
Pour la fonction unet_model, spécifiez la forme d'entrée, le nombre de filtres et le nombre de classes (23 dans ce cas).

Pour la première moitié du modèle :
*Commencez par un bloc conv qui prend les entrées du modèle et le nombre de filtres.
*Ensuite, enchaînez le premier élément de sortie de chaque bloc à l'entrée du bloc convolutif suivant.
*Ensuite, doublez le nombre de filtres à chaque étape.
À partir de conv_block4, ajoutez un abandon de 0,3.
*Pour le conv_block final, réglez à nouveau dropout sur 0,3 et désactivez la mise en commun maximale.

---
Pour la seconde mi-temps :

*Utilisez cblock5 comme expansive_input et cblock4 comme contractive_input, avec n_filters * 8. C'est votre couche de goulot d'étranglement.

*Chaînez la sortie du bloc précédent en expansive_input et la sortie correspondante du bloc contractif.

*Notez que vous devez utiliser le deuxième élément du bloc contractif avant la couche max pooling.

*A chaque étape, utilisez la moitié du nombre de filtres du bloc précédent
conv9 est une couche Conv2D avec activation ReLU, initialiseur normal He, même rembourrage(padding).

*Enfin, conv10 est un Conv2D qui prend le nombre de classes comme filtre, une taille de noyau de 1 et le "même" rembourrage. La sortie de conv10 est la sortie de votre modèle.




In [ ]:
# UNQ_C3
# GRADED FUNCTION: unet_model
def unet_model(input_size=(96, 128, 3), n_filters=32, n_classes=23):
    """
    Unet model
    
    Arguments:
        input_size -- Input shape 
        n_filters -- Number of filters for the convolutional layers
        n_classes -- Number of output classes
    Returns: 
        model -- tf.keras.Model
    """
    inputs = Input(input_size)
    # Contracting Path (encoding)
    # Add a conv_block with the inputs of the unet_ model and n_filters
    ### START CODE HERE
    cblock1 = conv_block(inputs, n_filters)
    # Chain the first element of the output of each block to be the input of the next conv_block. 
    # Double the number of filters at each new step
    cblock2 = conv_block(cblock1[0], n_filters*2)
    cblock3 = conv_block(cblock2[0], n_filters*4)
    cblock4 = conv_block(cblock3[0], n_filters*8, dropout_prob=0.3) # Include a dropout of 0.3 for this layer
    # Include a dropout of 0.3 for this layer, and avoid the max_pooling layer
    cblock5 = conv_block(cblock4[0], n_filters*16, dropout_prob=0.3, max_pooling=False) 
    ### END CODE HERE
    
    # Expanding Path (decoding)
    # Add the first upsampling_block.
    # Use the cblock5[0] as expansive_input and cblock4[1] as contractive_input and n_filters * 8
    ### START CODE HERE
    ublock6 = upsampling_block(cblock5[0], cblock4[1],  n_filters * 8)
    # Chain the output of the previous block as expansive_input and the corresponding contractive block output.
    # Note that you must use the second element of the contractive block i.e before the maxpooling layer. 
    # At each step, use half the number of filters of the previous block 
    ublock7 = upsampling_block(ublock6, cblock3[1],  n_filters * 4)
    ublock8 = upsampling_block(ublock7, cblock2[1],  n_filters * 2)
    ublock9 = upsampling_block(ublock8, cblock1[1],  n_filters)
    ### END CODE HERE

    conv9 = Conv2D(n_filters,
                 3,
                 activation='relu',
                 padding='same',
                 kernel_initializer='he_normal')(ublock9)

    # Add a Conv2D layer with n_classes filter, kernel size of 1 and a 'same' padding
    ### START CODE HERE
    conv10 = Conv2D(n_classes, 1, padding='same')(conv9)
    ### END CODE HERE
    
    model = tensorflow.keras.Model(inputs=inputs, outputs=conv10)

    return model

In [ ]:
import tensorflow

In [ ]:
unet_model_output = [['InputLayer', [(None, 96, 128, 3)], 0],
['Conv2D', (None, 96, 128, 32), 896, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 96, 128, 32), 9248, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 48, 64, 32), 0, (2, 2)],
['Conv2D', (None, 48, 64, 64), 18496, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 48, 64, 64), 36928, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 24, 32, 64), 0, (2, 2)],
['Conv2D', (None, 24, 32, 128), 73856, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 24, 32, 128), 147584, 'same', 'relu', 'HeNormal'],
['MaxPooling2D', (None, 12, 16, 128), 0, (2, 2)],
['Conv2D', (None, 12, 16, 256), 295168, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 12, 16, 256), 590080, 'same', 'relu', 'HeNormal'],
['Dropout', (None, 12, 16, 256), 0, 0.3],
['MaxPooling2D', (None, 6, 8, 256), 0, (2, 2)],
['Conv2D', (None, 6, 8, 512), 1180160, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 6, 8, 512), 2359808, 'same', 'relu', 'HeNormal'],
['Dropout', (None, 6, 8, 512), 0, 0.3],
['Conv2DTranspose', (None, 12, 16, 256), 1179904],
['Concatenate', (None, 12, 16, 512), 0],
['Conv2D', (None, 12, 16, 256), 1179904, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 12, 16, 256), 590080, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 24, 32, 128), 295040],
['Concatenate', (None, 24, 32, 256), 0],
['Conv2D', (None, 24, 32, 128), 295040, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 24, 32, 128), 147584, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 48, 64, 64), 73792],
['Concatenate', (None, 48, 64, 128), 0],
['Conv2D', (None, 48, 64, 64), 73792, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 48, 64, 64), 36928, 'same', 'relu', 'HeNormal'],
['Conv2DTranspose', (None, 96, 128, 32), 18464],
['Concatenate', (None, 96, 128, 64), 0],
['Conv2D', (None, 96, 128, 32), 18464, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 96, 128, 32), 9248, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 96, 128, 32), 9248, 'same', 'relu', 'HeNormal'],
['Conv2D', (None, 96, 128, 23), 759, 'same', 'linear', 'GlorotUniform']]

In [ ]:
from google.colab import output


In [ ]:
img_height = 96
img_width = 128
num_channels = 3

unet = unet_model((img_height, img_width, num_channels))
comparator(summary(unet), unet_model_output)

In [ ]:
unet = unet_model((img_height, img_width, num_channels))


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Dense
from keras.models import Model


3.5 - Définir les dimensions du modèle

In [ ]:
img_height = 96
img_width = 128
num_channels = 3

unet = unet_model((img_height, img_width, num_channels))

Découvrez le résumé du modèle ci-dessous

In [ ]:
unet.summary()


3.6 - Fonction de perte
En segmentation sémantique, il faut autant de masques que de classes d'objets. Dans l'ensemble de données que vous utilisez, chaque pixel de chaque masque s'est vu attribuer une probabilité entière unique d'appartenir à une certaine classe, de 0 à num_classes-1. La classe correcte est la couche avec la probabilité la plus élevée.

Ceci est différent de l'entropie croisée catégorique, où les étiquettes doivent être codées à chaud (seulement des 0 et des 1). Ici, vous utiliserez une entropie croisée catégorique clairsemée comme fonction de perte, pour effectuer une prédiction multiclasse au niveau des pixels. L'entropie croisée catégorielle clairsemée est plus efficace que les autres fonctions de perte lorsque vous avez affaire à de nombreuses classes.

In [ ]:

tf.keras.utils.plot_model(unet, show_shapes=True)

In [ ]:
unet.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
# Let's install Tensorflow 2.8.0:
!pip install -q tensorflow==2.8.0-rc0

# And verify that it is now in its latest version:
import tensorflow as tf
print(tf.__version__)

Gestion des jeux de données :
Ci-dessous, définissez une fonction qui vous permet d'afficher à la fois une image d'entrée et sa vérité terrain : le vrai masque. Le vrai masque est ce à quoi votre sortie de modèle formé vise à se rapprocher le plus possible.

In [ ]:
def display(display_list):
    plt.figure(figsize=(16, 16))

    title = ['Input Image', 'True Mask', 'Predicted Mask']

    for i in range(len(display_list)):
        plt.subplot(1, len(display_list), i+1)
        plt.title(title[i])
        plt.imshow(tf.keras.preprocessing.image.array_to_img(display_list[i]))
        plt.axis('off')
    plt.show()

In [ ]:
for image, mask in image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

In [ ]:
for image, mask in processed_image_ds.take(1):
    sample_image, sample_mask = image, mask
    print(mask.shape)
display([sample_image, sample_mask])

4 - Former le modèle

In [ ]:
EPOCHS = 40
VAL_SUBSPLITS = 5
BUFFER_SIZE = 500
BATCH_SIZE = 32
processed_image_ds.batch(BATCH_SIZE)
train_dataset = processed_image_ds.cache().shuffle(BUFFER_SIZE).batch(BATCH_SIZE)
print(processed_image_ds.element_spec)
model_history = unet.fit(train_dataset, epochs=EPOCHS)

In [ ]:
# serialize model to JSON
model_json = unet.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)

In [ ]:
# serialize weights to HDF5
unet.save_weights("model.h5")
print("Saved model to disk")
 

4.1 - Créer des masques prédits
Maintenant, définissez une fonction qui utilise tf.argmax dans l'axe du nombre de classes pour renvoyer l'index avec la plus grande valeur et fusionner la prédiction en une seule image :

In [ ]:
def create_mask(pred_mask):
    pred_mask = tf.argmax(pred_mask, axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    return pred_mask[0]

4.2 - Précision du modèle de tracé
Voyons comment votre modèle s'en est sorti !

In [ ]:
plt.plot(model_history.history["accuracy"])


4.3 - Afficher les prediction
Ensuite, vérifiez vos masques prédits par rapport au vrai masque et à l'image d'entrée d'origine :

In [ ]:
def show_predictions(dataset=None, num=1):
    """
    Displays the first image of each of the num batches
    """
    if dataset:
        for image, mask in dataset.take(num):
            pred_mask = unet.predict(image)
            display([image[0], mask[0], create_mask(pred_mask)])
    else:
        display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])

In [ ]:
show_predictions(train_dataset, 6)


In [ ]:
def create_mask(pred_mask):
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0]


In [ ]:
def show_predictions(dataset=None, num=1):
  if dataset:
    for image, mask in dataset.take(num):
      pred_mask = unet.predict(image)
      display([image[0], mask[0], create_mask(pred_mask)])
  else:
    display([sample_image, sample_mask,
             create_mask(unet.predict(sample_image[tf.newaxis, ...]))])


In [ ]:
show_predictions()

In [ ]:
# Prediction Utilities

def get_test_image_and_annotation_arrays():
  '''
  Unpacks the test dataset and returns the input images and segmentation masks
  '''

  ds = test_dataset.unbatch()
  ds = ds.batch(info.splits['test'].num_examples)
  
  images = []
  y_true_segments = []

  for image, annotation in ds.take(1):
    y_true_segments = annotation.numpy()
    images = image.numpy()
  
  y_true_segments = y_true_segments[:(info.splits['test'].num_examples - (info.splits['test'].num_examples % BATCH_SIZE))]
  
  return images[:(info.splits['test'].num_examples - (info.splits['test'].num_examples % BATCH_SIZE))], y_true_segments


def create_mask(pred_mask):
  '''
  Creates the segmentation mask by getting the channel with the highest probability. Remember that we
  have 3 channels in the output of the UNet. For each pixel, the predicition will be the channel with the
  highest probability.
  '''
  pred_mask = tf.argmax(pred_mask, axis=-1)
  pred_mask = pred_mask[..., tf.newaxis]
  return pred_mask[0].numpy()


def make_predictions(image, mask, num=1):
  '''
  Feeds an image to a model and returns the predicted mask.
  '''

  image = np.reshape(image,(1, image.shape[0], image.shape[1], image.shape[2]))
  pred_mask = unet.predict(image)
  pred_mask = create_mask(pred_mask)

  return pred_mask 